1. Посмотреть распределение аллофонов:

1.0. Определить группы контекстов  
1.1. Прочитать файлы  
1.2. Найти аллофоны (B1)  
1.2.1. Сохранить аллофон с соседями  

In [23]:
import re

In [ ]:
left_vowel_contexts = {

}

In [2]:
from itertools import product
letters = "GBRY"
nums = "1234"
levels = [ch + num for num, ch in product(nums, letters)]
level_codes = [2 ** i for i in range(len(levels))]
code_to_level = {i: j for i, j in zip(level_codes, levels)}
level_to_code = {j: i for i, j in zip(level_codes, levels)}

In [3]:
def read_seg(filename: str, encoding: str = "utf-8-sig") -> tuple[dict, list[dict]]:
    with open(filename, encoding=encoding) as f:
        lines = [line.strip() for line in f.readlines()]

    # найдём границы секций в списке строк:
    header_start = lines.index("[PARAMETERS]") + 1
    data_start = lines.index("[LABELS]") + 1

    # прочитаем параметры
    params = {}
    for line in lines[header_start:data_start - 1]:
        key, value = line.split("=")
        params[key] = int(value)

    # прочитаем метки
    labels = []
    for line in lines[data_start:]:
        # если в строке нет запятых, значит, это не метка и метки закончились
        if line.count(",") < 2:
            break
        pos, level, name = line.split(",", maxsplit=2)
        label = {
            "position": int(pos) // params["BYTE_PER_SAMPLE"] // params["N_CHANNEL"],
            "level": code_to_level[int(level)],
            "name": name
        }
        labels.append(label)
    return params, labels

In [1]:
import struct

In [4]:
def read_sbl(filename):
    samplerate = 22050
    sampwidth = 2
    num_channels = 1
    sampwidth_to_char = {1: "c", 2: "h", 4: "i"}

    with open(filename, "rb") as f:
        raw_signal = f.read()

    num_samples = len(raw_signal) // sampwidth
    fmt = str(num_samples) + sampwidth_to_char[sampwidth]
    signal = struct.unpack(fmt, raw_signal)
    return signal

In [21]:
def cut_sbl_by_labels(sbl_file, seg_file):
    signal = read_sbl(sbl_file)
    params, labels = read_seg(seg_file)
    labels_contexts = get_contexts_and_start_end(labels)
    signal_cut = {}
    for label in labels_contexts:
        signal_cut[label[:3]] = signal[label[3]:label[4]]
    return signal_cut

In [47]:
from collections import defaultdict

In [48]:
def cut_sbl_by_labels_2_electric_boogaloo(sbl_file, seg_file):
    signal = read_sbl(sbl_file)
    params, labels = read_seg(seg_file)
    labels_contexts = get_contexts_and_start_end(labels)
    signal_cut = defaultdict(list)
    for label in labels_contexts:
        signal_cut[label[:3]].append(signal[label[3]:label[4]])
    return signal_cut

Нам понадобится файл cta_seg.zip

In [4]:
!unzip -q cta_seg.zip

In [5]:
import glob
files_B1 = glob.glob(r"D:\CORPRES\careva\cta\**\*.seg_B1", recursive=True)

In [25]:
res = cut_sbl_by_labels(files_B1[0].replace(".seg_B1", ".sbl"), files_B1[0])

In [26]:
list(res.keys())

[('start', 'j', 'rounded'),
 ('j', 'u0', 'palat'),
 ('vowels', "r'", 'unrounded'),
 ('dentals_palat', 'i4', 'dentals'),
 ('vowels', 't', 'sonorants'),
 ('consonants', "r'", 'unrounded'),
 ('dentals_palat', 'i0', 'labials'),
 ('vowels', 'f', 'unrounded'),
 ('labials', 'a4', 'dentals'),
 ('vowels', 'n', 'unrounded'),
 ('n', 'a4', 'labials'),
 ('vowels', 'f', 'end')]

In [28]:
from IPython.display import Audio
Audio(res[('vowels', 't', 'sonorants')], rate=22050)

In [29]:
allophones_dict = {}
for file in files_B1:
    allophones_dict.update(cut_sbl_by_labels(file.replace(".seg_B1", ".sbl"), file))

In [30]:
len(allophones_dict)

2609

In [49]:
allophones_dict_2 = defaultdict(list)
for file in files_B1:
    small_dict = cut_sbl_by_labels_2_electric_boogaloo(file.replace(".seg_B1", ".sbl"), file)
    for context, parts in small_dict.items():
        allophones_dict_2[context].extend(parts)

In [50]:
mean_durs = {}
for allophone in allophones_dict_2:
    mean_durs[allophone] = np.mean([len(i) for i in allophones_dict_2[allophone]])

In [54]:
allophones_dict_mean_durs = {}
for allophone in allophones_dict_2:
    allophones_dict_mean_durs[allophone] = min(allophones_dict_2[allophone], key=lambda x: abs(len(x) - mean_durs[allophone]))

In [33]:
def get_contexts_for_string(text):
    text = [""] + text.split(" ") + [""]
    text_context = []
    for l_1, l_2, l_3 in zip(text, text[1:], text[2:]):
        if l_2.startswith(tuple(vowels)):
            cur_class = "vowel"
        elif l_2 in consonants:
            cur_class = "consonant"
        else:
            continue
        left_context = re.sub("\d", "", l_1)
        sound2class_left = sound2class_cl if cur_class == "consonant" else sound2class_vl
        sound2class_right = sound2class_cr if cur_class == "consonant" else sound2class_vr
        left_class = sound2class_left.get(left_context, "???")
        right_context = re.sub("\d", "", l_3)
        right_class = sound2class_right.get(right_context, "???")
        text_context.append((left_class, l_2, right_class))
    return text_context

In [42]:
test_string = "a0 n n a4 m a2 l a1 d' e0 c"
test_contexts = get_contexts_for_string(test_string)

In [55]:
signal_res = []
for c in test_contexts:
    part = allophones_dict_mean_durs[c]
    signal_res.extend(part)

In [56]:
from IPython.display import Audio
Audio(signal_res, rate=22050)

In [57]:
import scipy.io.wavfile as wavfile
import numpy as np
wavfile.write("anna2.wav", 22050, np.asarray(signal_res, dtype=np.int16))

In [6]:
from collections import Counter

In [7]:
vowels = "i e a o u y".split()

In [8]:
permissible_allophones = [
    ('a1', 3748),
    ('a0', 3178),
    ('t', 3154),
    ('o0', 3128),
    ('i1', 3063),
    ('n', 2948),
    ('y4', 2699),
    ('i4', 2682),
    ('l', 2149),
    ('k', 2135),
    ('e0', 2111),
    ('v', 2035),
    ('r', 2017),
    ('s', 2015),
    ("n'", 1800),
    ('d', 1726),
    ('m', 1664),
    ('a4', 1614),
    ('p', 1612),
    ('e4', 1468),
    ("l'", 1432),
    ('i0', 1311),
    ('y1', 1272),
    ("s'", 1201),
    ("t'", 1144),
    ("r'", 1079),
    ('j', 1063),
    ('z', 1017),
    ('sh', 914),
    ('ch', 906),
    ('g', 859),
    ('b', 842),
    ('u0', 840),
    ('u1', 807),
    ('f', 785),
    ("m'", 767),
    ('u4', 750),
    ('zh', 737),
    ("v'", 663),
    ("d'", 648),
    ('y0', 636),
    ('h', 542),
    ('e1', 484),
    ('c', 368),
    ('a2', 365),
    ("k'", 307),
    ("b'", 260),
    ('o4', 243),
    ('sc', 233),
    ("p'", 228),
    ("z'", 178),
    ('o1', 147),
    ("g'", 95),
    ("f'", 78),
    ('H', 28),
    ("h'", 25),
    ('ch_', 16),
    ("zh'", 14),
    ('~', 5),
    ('CH', 3),
    ('C', 3),
]

In [9]:
consonants = [i[0] for i in permissible_allophones if i[0][0] not in vowels]

In [10]:
class2sound_vl = {
    "labials": "f v p b".split(),
    "labials_palat": "f' v' p' b'".split(),
    "dentals": "t d s z c".split(),
    "dentals_palat": "t' d' s' z' ch sc r' zh'".split(),
    "postalveolars": "sh zh ch_".split(),
    "velars": "k g h".split(),
    "velars_palat": "k' g' h'".split(),
    "start": [""]
}
class2sound_vl.update({i: [i] for i in "m m' n n' l l' r j a o u e i y".split()})

In [11]:
sound2class_vl = {j: i for i in class2sound_vl for j in class2sound_vl[i]}

In [13]:
class2sound_vr = {
    "labials": "p b m f v".split(),
    "dentals": "t d s z c C n r".split(),
    "postalveolars": "sh zh ch_".split(),
    "velars": "k g h H".split(),
    "palat": "f' v' p' b' t' d' s' z' ch sc r' zh' k' g' h' m' n' l' r'".split(),
    "end": [""]
}
class2sound_vr.update({i: [i] for i in "l j a o u e i y".split()})

In [14]:
sound2class_vr = {j: i for i in class2sound_vr for j in class2sound_vr[i]}

In [15]:
class2sound_cl = {
    "vowels": vowels,
    "consonants": consonants,
    "start": [""]
}

In [16]:
sound2class_cl = {j: i for i in class2sound_cl for j in class2sound_cl[i]}

In [17]:
class2sound_cr = {
    "rounded": "o u".split(),
    "unrounded": "i e a y".split(),
    "voiceless": "t k s p s' t' sh ch f h c k' sc p' f' h' ch_".split(),
    "sonorants": "n l v r n' m l' r' j m' v'".split(),
    "voiced": "d z g b zh d' b' z' g' H CH C".split(),
    "end": [""]
}

In [18]:
sound2class_cr = {j: i for i in class2sound_cr for j in class2sound_cr[i]}

In [ ]:
parasm, labels = read_seg("cta_seg/cta0001.seg_B1")

In [19]:
import re

In [20]:
def get_contexts_and_start_end(labels):
    labels = [{"position": 0, "name": ""}] + labels
    labels_context = []

    for l_1, l_2, l_3 in zip(labels, labels[1:], labels[2:]):
        if l_2["name"].startswith(tuple(vowels)):
            cur_class = "vowel"
        elif l_2["name"] in consonants:
            cur_class = "consonant"
        else:
            continue
        left_context = re.sub("\d", "", l_1["name"])
        sound2class_left = sound2class_cl if cur_class == "consonant" else sound2class_vl
        sound2class_right = sound2class_cr if cur_class == "consonant" else sound2class_vr
        left_class = sound2class_left.get(left_context, "???")
        right_context = re.sub("\d", "", l_3["name"])
        right_class = sound2class_right.get(right_context, "???")
        labels_context.append((left_class, l_2["name"], right_class, l_2["position"], l_3["position"]))
    return labels_context

In [24]:
labels_context

[('start', 'j', 'rounded'),
 ('j', 'u0', 'palat'),
 ('vowels', "r'", 'unrounded'),
 ('dentals_palat', 'i4', 'dentals'),
 ('vowels', 't', 'sonorants'),
 ('consonants', "r'", 'unrounded'),
 ('dentals_palat', 'i0', 'labials'),
 ('vowels', 'f', 'unrounded'),
 ('labials', 'a4', 'dentals'),
 ('vowels', 'n', 'unrounded'),
 ('n', 'a4', 'labials'),
 ('vowels', 'f', 'end')]

In [22]:
allophones = Counter()
for file in files_B1:
    params, labels = read_seg(file)
    phones = [l["name"] for l in labels if l["name"]]
    if "zh'" in phones:
        print(file)
    allophones.update(phones)

cta_seg/cta0779.seg_B1
cta_seg/cta0920.seg_B1
cta_seg/cta1003.seg_B1
cta_seg/cta0378.seg_B1
cta_seg/cta0005.seg_B1
cta_seg/cta0736.seg_B1
cta_seg/cta0784.seg_B1
cta_seg/cta0338.seg_B1
cta_seg/cta0663.seg_B1
cta_seg/cta0200.seg_B1


In [ ]:
allophones.most_common()